In [23]:
%load_ext autoreload
%autoreload 2

import pandas as pd

# Set display options to show more columns and wider output
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Don't truncate column contents

from IPython.display import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read Messeges

In [8]:
import json
import pandas as pd

# Specify the file path
file_path = "telegram_messages.json"  # Replace with the actual file path if different

# Read the JSON file
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert the JSON data to a Pandas DataFrame
df = pd.json_normalize(data)

# Display the DataFrame
df.head()

,channel,message_id,timestamp,message,media,metadata.sender_id,metadata.message_type
0,From_hebron,270263,2024-10-13 19:32:55,♦️إعلام العدو يتحدث عن حدث أمني صعب.,[],-1001478404564,Message
1,From_hebron,270262,2024-10-13 19:32:20,♦️ يسرائيل للو تسنزورا: إصابة قاعدة تدريب غولا...,[],-1001478404564,Message
2,From_hebron,270261,2024-10-13 19:31:11,محكمة الاحتلال تُمدد الحكم على الأسير سليم الر...,"[{'media_type': 'MessageMediaPhoto', 'media_id...",-1001478404564,Message
3,From_hebron,270260,2024-10-13 19:29:24,♦️المتحدثة باسم الشرطة الصهيونيه : المتظاهرون ...,"[{'media_type': 'MessageMediaDocument', 'media...",-1001478404564,Message
4,From_hebron,270259,2024-10-13 19:29:07,♦️ القسام يعلن تفجير عين نفق في قوة اسرائيلية ...,[],-1001478404564,Message


In [10]:
from cost_calculator import CostCalculator

# Initialize the cost calculator with a model that will be resolved to the closest match
cost_calculator = CostCalculator('gpt-4o-2024-08-06')

# Calculate costs
text_prompt = "Hello, how are you today?"
text_completion = "I am doing well, thank you for asking!"

# Calculate prompt cost and completion cost
prompt_cost = cost_calculator.calculate_prompt_cost(text_prompt, has_image=True)
completion_cost = cost_calculator.calculate_completion_cost(text_completion)

# Output the results
print(f"Prompt cost: ${prompt_cost:.6f}")
print(f"Completion cost: ${completion_cost:.6f}")

Using closest match for encoding: 'gpt-4o' for input model 'gpt-4o-2024-08-06'
Resolved model for encoding: 'gpt-4o'
Using pricing model: 'gpt-4o-2024-08-06'
Prompt cost: $0.002118
Completion cost: $0.000100


In [24]:
import pandas as pd
from typing import Optional, Dict
from pydantic import BaseModel
from openai_client import OpenAIClient
from tqdm import tqdm

class TranslationResponse(BaseModel):
    """Response model for translations."""
    original: str
    hebrew: str
    english: str

class TelegramMessageProcessor:
    """Processor class to handle Telegram message translation."""
    
    def __init__(self, client: OpenAIClient):
        self.client = client

    def translate_message(self, message: str) -> Optional[TranslationResponse]:
        """Translate a Telegram message into both Hebrew and English."""
        response = self.client.chat(
            system_message="Translate this message into both Hebrew and English.",
            user_message=message,
            response_format=TranslationResponse  # Using the custom response format
        )

        if response and response.choices and hasattr(response.choices[0].message, 'parsed'):
            return response.choices[0].message.parsed  # Parsed translation content
        return None

class MainProcessor:
    """Class to manage the full pipeline: reading messages, translating, and updating DataFrame."""
    
    def __init__(self, processor: TelegramMessageProcessor, df: pd.DataFrame):
        self.processor = processor
        self.df = df.copy()  # Avoid modifying the original df

        # Prepare new columns for translations
        self.df['hebrew_translation'] = None
        self.df['english_translation'] = None

    def run(self) -> pd.DataFrame:
        """Run the full pipeline, translating each message and returning an updated DataFrame."""
        # Use tqdm to create a progress bar for the loop
        for index, row in tqdm(self.df.iterrows(), total=len(self.df), desc="Translating messages"):
            message = row['message']
            translation = self.processor.translate_message(message)
            
            if translation:
                # Store translations in corresponding DataFrame columns
                self.df.at[index, 'hebrew_translation'] = translation.hebrew
                self.df.at[index, 'english_translation'] = translation.english
            else:
                print(f"No translation found for message_id {row['message_id']}.")
        
        # Return the updated DataFrame with translations
        return self.df

if __name__ == "__main__":
    # Specify the file path
    file_path = "telegram_messages.json"  # Replace with the actual file path if different
    
    # Read the JSON file
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # Convert the JSON data to a Pandas DataFrame
    df = pd.json_normalize(data)

    # Initialize the OpenAI client and processor
    openai_client = OpenAIClient(show_prices=False)
    message_processor = TelegramMessageProcessor(openai_client)
    main_processor = MainProcessor(message_processor, df)

    # Run the pipeline and get the updated DataFrame
    updated_df = main_processor.run()

    # Display the updated DataFrame with translations
    display(updated_df[['message', 'hebrew_translation', 'english_translation']])


Using closest match for encoding: 'gpt-4o' for input model 'gpt-4o-2024-08-06'
Resolved model for encoding: 'gpt-4o'
Using pricing model: 'gpt-4o-2024-08-06'


Translating messages: 100%|████████████████████████████████████████████████████| 25/25 [00:53<00:00,  2.15s/it]


,channel,message_id,timestamp,message,media,metadata.sender_id,metadata.message_type,hebrew_translation,english_translation
0,From_hebron,270263,2024-10-13 19:32:55,♦️إعلام العدو يتحدث عن حدث أمني صعب.,[],-1001478404564,Message,♦️התקשורת של האויב מדברת על אירוע ביטחוני קשה.,♦️Enemy media talks about a difficult security incident.
1,From_hebron,270262,2024-10-13 19:32:20,♦️ يسرائيل للو تسنزورا: إصابة قاعدة تدريب غولاني \nبطائرة\n مسيرة تابعة للحزب في عكا قبل قليل,[],-1001478404564,Message,♦️ ישראל לו צנזורה: פגיעת בסיס האימונים של גולני ברחפן של המפלגה בעכו לפני זמן קצר,♦️ Yisrael Lo Tenzura: Golani training base hit by a party's drone in Acre moments ago
2,From_hebron,270261,2024-10-13 19:31:11,محكمة الاحتلال تُمدد الحكم على الأسير سليم الرجوب من سكان حي كريسة بمدينة دورا للمرة الثالثة على التوالي لمدة ستة أشهر إضافية.,"[{'media_type': 'MessageMediaPhoto', 'media_id': 270261}]",-1001478404564,Message,"בית המשפט של הכיבוש מאריך את גזר הדין של האסיר סלים רג'וב, תושב שכונת כריסי בעיר דורא, בפעם השלישית ברציפות לעוד שישה חודשים נוספים.","The occupation court extends the sentence of prisoner Salim Rajoub, a resident of Karisa neighborhood in the city of Dora, for the third consecutive time for an additional six months."
3,From_hebron,270260,2024-10-13 19:29:24,♦️المتحدثة باسم الشرطة الصهيونيه : المتظاهرون ضد الحكومة عرضوا حياة ضباط الشرطة للخطر أمس عندما تعمدوا إشعال حريق كبير على بعد بوصات منهم.,"[{'media_type': 'MessageMediaDocument', 'media_id': 270260}]",-1001478404564,Message,הדוברת של המשטרה הציונית: המפגינים נגד הממשלה סיכנו את חיי השוטרים אתמול כשהציתו באופן מכוון אש גדולה במרחק של סנטימטרים מהם.,The spokesperson for the Zionist police: The protesters against the government endangered the lives of police officers yesterday when they deliberately set a large fire inches away from them.
4,From_hebron,270259,2024-10-13 19:29:07,♦️ القسام يعلن تفجير عين نفق في قوة اسرائيلية راجلة شمال مدينة بيت لاهيا شمال القطاع,[],-1001478404564,Message,הקסאם מכריז על פיצוץ עין מנהרה בכוח ישראלי רגלי צפונית לעיר בית לאהיה בצפון הרצועה,"Al-Qassam announces detonating a tunnel entrance against an Israeli foot patrol north of Beit Lahia, in the northern part of the Strip."
5,From_hebron,270258,2024-10-13 19:25:48,🛑اعتراض الطائرة بدون طيار في سماء كريات قبل قليل,"[{'media_type': 'MessageMediaDocument', 'media_id': 270258}]",-1001478404564,Message,"🛑יירוט הכטב""מ בשמי קרית לפני זמן קצר",🛑Interception of a drone in the sky above Kiryat shortly ago
6,From_hebron,270257,2024-10-13 19:19:43,"🚫🟠🟠 🟢 كتائب القسام تستهدف ناقلة جند صهيونية بقذيفة ""تاندوم"" شمال مدينة بيت لاهيا شمال القطاع",[],-1001478404564,Message,"גדודי אל-קסאם תוקפים רכב משוריין ישראלי בטיל ""טנדום"" בצפון בית להיא בצפון הרצועה","Al-Qassam Brigades target an Israeli armored vehicle with a ""Tandem"" missile north of Beit Lahia in the northern Gaza Strip."
7,From_hebron,270256,2024-10-13 19:18:08,♦️بالفيديو | كلمة سماحة الأمين العام الشهيد الي ( المجاهدين خلال إحدى المناورات العسكرية,"[{'media_type': 'MessageMediaDocument', 'media_id': 270256}]",-1001478404564,Message,בווידאו | נאומו של המזכיר הכללי השהיד ללוחמים באחת התמרונים הצבאיים.,In the video | The speech of the martyred Secretary General to the fighters during a military exercise.
8,From_hebron,270255,2024-10-13 19:15:36,♦️عاجل| حزب الله يشن هجوماً مركباً بالطائرات المسيّرة والصواريخ على شمال فلسطين المحتلة.,[],-1001478404564,Message,"דחוף: חיזבאללה משגר מתקפה מורכבת במל""טים וטילים על צפון פלסטין הכבושה.",Urgent: Hezbollah launches a complex attack with drones and missiles on Northern occupied Palestine.
9,From_hebron,270254,2024-10-13 19:15:13,♦️التقارير الأولية دون رقابة: \nإصابة مستوطن نتيجة سقوط شظايا في منطقة كريات شمونة.,[],-1001478404564,Message,♦️דיווחים ראשוניים ללא צנזורה: \nפצוע מתיישב כתוצאה מנפילת רסיסים באזור קריית שמונה.,♦️Initial uncensored reports: \nA settler was injured due to shrapnel falling in the area of Kiryat Shmona.


In [25]:
display(updated_df[['message', 'hebrew_translation', 'english_translation']])

,message,hebrew_translation,english_translation
0,♦️إعلام العدو يتحدث عن حدث أمني صعب.,♦️התקשורת של האויב מדברת על אירוע ביטחוני קשה.,♦️Enemy media talks about a difficult security incident.
1,♦️ يسرائيل للو تسنزورا: إصابة قاعدة تدريب غولاني \nبطائرة\n مسيرة تابعة للحزب في عكا قبل قليل,♦️ ישראל לו צנזורה: פגיעת בסיס האימונים של גולני ברחפן של המפלגה בעכו לפני זמן קצר,♦️ Yisrael Lo Tenzura: Golani training base hit by a party's drone in Acre moments ago
2,محكمة الاحتلال تُمدد الحكم على الأسير سليم الرجوب من سكان حي كريسة بمدينة دورا للمرة الثالثة على التوالي لمدة ستة أشهر إضافية.,"בית המשפט של הכיבוש מאריך את גזר הדין של האסיר סלים רג'וב, תושב שכונת כריסי בעיר דורא, בפעם השלישית ברציפות לעוד שישה חודשים נוספים.","The occupation court extends the sentence of prisoner Salim Rajoub, a resident of Karisa neighborhood in the city of Dora, for the third consecutive time for an additional six months."
3,♦️المتحدثة باسم الشرطة الصهيونيه : المتظاهرون ضد الحكومة عرضوا حياة ضباط الشرطة للخطر أمس عندما تعمدوا إشعال حريق كبير على بعد بوصات منهم.,הדוברת של המשטרה הציונית: המפגינים נגד הממשלה סיכנו את חיי השוטרים אתמול כשהציתו באופן מכוון אש גדולה במרחק של סנטימטרים מהם.,The spokesperson for the Zionist police: The protesters against the government endangered the lives of police officers yesterday when they deliberately set a large fire inches away from them.
4,♦️ القسام يعلن تفجير عين نفق في قوة اسرائيلية راجلة شمال مدينة بيت لاهيا شمال القطاع,הקסאם מכריז על פיצוץ עין מנהרה בכוח ישראלי רגלי צפונית לעיר בית לאהיה בצפון הרצועה,"Al-Qassam announces detonating a tunnel entrance against an Israeli foot patrol north of Beit Lahia, in the northern part of the Strip."
5,🛑اعتراض الطائرة بدون طيار في سماء كريات قبل قليل,"🛑יירוט הכטב""מ בשמי קרית לפני זמן קצר",🛑Interception of a drone in the sky above Kiryat shortly ago
6,"🚫🟠🟠 🟢 كتائب القسام تستهدف ناقلة جند صهيونية بقذيفة ""تاندوم"" شمال مدينة بيت لاهيا شمال القطاع","גדודי אל-קסאם תוקפים רכב משוריין ישראלי בטיל ""טנדום"" בצפון בית להיא בצפון הרצועה","Al-Qassam Brigades target an Israeli armored vehicle with a ""Tandem"" missile north of Beit Lahia in the northern Gaza Strip."
7,♦️بالفيديو | كلمة سماحة الأمين العام الشهيد الي ( المجاهدين خلال إحدى المناورات العسكرية,בווידאו | נאומו של המזכיר הכללי השהיד ללוחמים באחת התמרונים הצבאיים.,In the video | The speech of the martyred Secretary General to the fighters during a military exercise.
8,♦️عاجل| حزب الله يشن هجوماً مركباً بالطائرات المسيّرة والصواريخ على شمال فلسطين المحتلة.,"דחוף: חיזבאללה משגר מתקפה מורכבת במל""טים וטילים על צפון פלסטין הכבושה.",Urgent: Hezbollah launches a complex attack with drones and missiles on Northern occupied Palestine.
9,♦️التقارير الأولية دون رقابة: \nإصابة مستوطن نتيجة سقوط شظايا في منطقة كريات شمونة.,♦️דיווחים ראשוניים ללא צנזורה: \nפצוע מתיישב כתוצאה מנפילת רסיסים באזור קריית שמונה.,♦️Initial uncensored reports: \nA settler was injured due to shrapnel falling in the area of Kiryat Shmona.
